<img style="float: center;" src="images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>


<center> Julia Lane, Benjamin Feder, Angela Tombari, Ekaterina Levitskaya, Tian Lou, Lina Osorio-Copete. </center> 

## Data Visualization in R

#### Introduction

In this notebook you will learn how to use different visualization methods in R to explore and analyze your data. We will also talk about proper annotation of graphs in order to clearly and accurately communicate your results.

We will cover the following methods:
- **Histogram** 
(visualizing distributions, continuous variables)
- **Bar plot**
(visualizing relationships between numerical and categorical variables)
- **Small multiples**
(using a series of mini-graphs to compare information by different groups)
- **Heatmap** 
(adding highlights to your data with color-coding)
- **Geographic heatmap**
(showing regional differences in your data)

For all visualizations we are going to use an R package called `ggplot2` (`ggplot2` is included in the `tidyverse` suite of packages). The syntax of `ggplot2` in most cases stays the same:

- you always start with `ggplot()` <br>
- then, supply a dataset and aesthetic mapping with x and/or y variables, like this: `ggplot(dataset, aes(x = ..., y = ...)` <br>
- and then you can add layers using `+` <br>
for example, <br>
`ggplot(dataset, aes(x = ... , y = ...) + geom_histogram()` to create a histogram, or <br>
`ggplot(dataset, aes(x = ... , y = ...) + geom_histogram() + labs(title = 'My plot title')` to create a histogram and add a title for the graph, and so on.

In this notebook we will visualize the following examples for our cohort of Kentucky graduates in the 2013 academic year (defined in the Data Exploration notebook):

- Education experience:
    - **Number of graduates by institution** (boxplot)
    - **Number of graduates by origin county** (geographic heatmap)
    - **Percentages of graduates who received their primary degrees within the seven major groups by Applachian status** (bar plot)
    

- Employment outcomes:
    - **Distribution of quarterly wages by degree rank** (small multiples, histogram)
    - **Percentage of graduates in five key sectors of employment by Appalachian status** (bar plot)
    - **Employment patterns by quarters** (heatmap)

### R Setup

Let's start by importing necessary R libraries and connecting to the database.

In [ ]:
#database interaction imports
library(DBI)
library(RPostgreSQL)

# for data manipulation/visualization
library(tidyverse)
library(lubridate)
library(sf)

In [ ]:
# create an RPostgreSQL driver
drv <- dbDriver("PostgreSQL")

# connect to the database
con <- dbConnect(drv,dbname = "postgresql://stuffed.adrf.info/appliedda")

Now that we can properly connect to our `appliedda` database, let's read in the permanent table `cohort` from the `ada_ky_20` schema. As a reminder, `cohort` contains the cohort we used in the Data Exploration notebook.

In [ ]:
# read in cohort table as df
qry <- "
select * 
from ada_ky_20.cohort
"
df <- dbGetQuery(con, qry)

In [ ]:
# see df
glimpse(df)

## Learning About our Graduates

In the Data Exploration notebook, we found numerical summaries to answer a variety of questions that ultimately helped us understand our cohort a bit better. As you will see, some of these numerical summaries can be better conveyed via visualizations. In this section, we will introduce three visualizations:

- A boxplot of the number of graduates by institution (`geom_boxplot()`)
- A heatmap of origin counties of our graduates (`geom_sf()`)
- Degree breakdowns by Appalachian status (`geom_bar()`)

We will start by reusing the code from before to generate these numerical summaries, and then pipe (`%>%`) at least some portion of the resulting dataframe into a `ggplot()` string of functions to create the resulting visualization.

### Number of graduates by institution

Based on our analysis of the most popular institutions by graduation, it seems as though a handful of insitutions in Kentucky tend to account for a significant proportion of its entire postsecondary graduating class. Let's further analyze that hypothesis by plotting a boxplot of the amount of graduates in our cohort.

As a reminder, the code cell below uses the exact code we used to get to this calculation in the Data Exploration notebook.

In [ ]:
# counts by institution
df %>%
    count(kpeds_instname) %>%
    arrange(desc(n)) %>%
    head(10)

We can use a boxplot to inspect the overall distribution of the number of graduates for each institution. We can simply take the output from `count(kpeds_instname)` and use it as the first argument in our `ggplot()` call.

> Note: We can ensure that we are only plotting one group by setting `x=""` inside the `aes()` function. You will also see a manual line break to wrap the title: `\n`.

In [ ]:
# Full code for the plot
df %>%
    count(kpeds_instname) %>%
    ggplot(aes(x="", y=n)) + 
    geom_boxplot() +
    labs(
        title = 'The majority of institutions graduated around 1,000 students \n in 2013',
        y = 'Number of graduates',
        x='',
        caption = 'Source: KPEDS data'
    ) +
    theme_minimal()

This visualization shows us that there seem to be a handful of outlier institutions in terms of number of graduates in the 2013 academic year.  After establishing this baseline, we can see if there is any variation in the number of graduates by institution type. 

As you may recall from the Data Exploration notebook, we can either use `kpeds_sector` or `kpeds_sector_code`. Let's look at both of them together.

In [ ]:
# see counts by kpeds_sector and kpeds_sector_code
df %>%
    count(kpeds_sector, kpeds_sector_code)

You may have remembered that there are two potential `kpeds_sector` values that relate to the same sector code, with AIKCU and 4 year independent both corresponding to `kpeds_sector_code == 3`. Let's update `df` so that all AIKCU institutions are now referred to as 4 year independent ones. 

Once sector names are cleaned up, we will look at the distribution of each institution's number of graduates by institution type.

In [ ]:
# update kpeds_sector
df <- df %>%
    mutate(kpeds_sector = ifelse(kpeds_sector == "AIKCU", "4 year independent", kpeds_sector))

Now let's use almost identical code from the first boxplot, except with `count(kpeds_instname, kpeds_sector)`.

In [ ]:
# counts by institution
# switch to just using labs and theme here
df %>%
    count(kpeds_instname, kpeds_sector) %>%
    ggplot(aes(x=kpeds_sector, y=n)) + 
    geom_boxplot() +
    labs(
        title = 'The number of graduates differs by institution type',
        y = 'Number of graduates',
        x='Institution Type',
        caption = 'Source: KPEDS data'
    ) +
    theme_minimal()

<font color=red><h3> Checkpoint 1: New Aesthetics </h3></font> 

Recreate the same bar plot, except instead having a blank x-axis and the color differing by the institution type. Which of the two options do you prefer?

> You can dictate the color as a separate argument inside `aes()`.

In [ ]:
# same boxplot except institution type differs based on color


### Number of graduates by origin county

What if we wanted to show regional differences in the number of graduates by county using a map? A heatmap is a powerful visualization tool which allows for easy comparison and communication of regional differences to external audiences.

We can leverage an available public table in our database with geographic coordinates of counties, as well as the `sf` package, which allows us to read in the geographic location information in one line of code (using `st_read` function) and prepare coordinate information for plotting. 

Before we do so, we will generate our base table of graduates by origin county.

>We have already pulled our graduates (`df`) into our working environment. We still need enrollment information, which contains postsecondary student location of origin (`kpeds_enrollments.kpeds_statecountryoforigin` and `kpeds_enrollments.kpeds_countyoforigin`).  We will pull this information into our working environment as a data frame, `enroll`.

In [ ]:
# read kpeds_enrollments into R
qry <- "
select *
from kystats_2020.kpeds_enrollments
where kpeds_semesteryear in (2012, 2013)
"
enroll <- dbGetQuery(con, qry)

In the code cell below, since we do not want any entries with `NA` values as the county and we want to make sure we can match each county to our table in the `public` schema, we will make sure all of the county names are in lowercase by using the `tolower()` function. Additionally, since an individual may have multiple enrollments within the time span of `enroll`, after we match all potential enrollments to each member of our cohort, we will keep only one enrollment entry, preferably (as indicated in the `arrange()` call) from the enrollment institution that matches the institution of graduation for each individual.

In [ ]:
# find counts by origin county in Kentucky
# save output into a new data frame (cnty) in our working environment

cnty <- df %>% 
    left_join(enroll, "coleridge_id") %>%
    # added tolower because a few records have KENTUCKY instead of Kentucky
    filter(tolower(kpeds_statecountryoforigin) == "kentucky", !is.na(kpeds_countyoforigin)) %>%
    # some coleridge_ids became duplicated by join, so keep only 1, preferring 1 from graduating institution if available
    arrange(coleridge_id, desc(kpeds_institution.x == kpeds_institution.y)) %>%
    distinct(coleridge_id, .keep_all = TRUE) %>%
    # now can count graduates by county of origin
    count(kpeds_countyoforigin) %>%
    mutate(kpeds_countyoforigin = tolower(kpeds_countyoforigin))

# see cnty
head(cnty)

In [ ]:
nrow(df)      # original coleridge_id count
sum(cnty$n)  # record count after enrollment join:  decrease due to filter for in-state only and dropping graduates with no enrollment record

Now that we have our table of graduate counts by origin county, We will use the `st_read` function in the `sf` package to read in the necessary geographic information for Kentucky: county codes, county names, polygons, and centroids of those polygons.

In [ ]:
# Get county codes, county names, polygons, and centroids of those polygons for Kentucky (fips code = '21')
qry <- 
"SELECT countyfp as county, LOWER(name) as name, geom, ST_X(ST_Centroid(geom)) as long, ST_Y(ST_Centroid(geom)) as lat
FROM public.tl_2016_us_county
WHERE statefp = '21'
"
#read into R as df
counties <- st_read(con,query=qry)

In [ ]:
# see counties
head(counties)

We can now join `counties` and `cnty`.

In [ ]:
# join counties and cnty
counties <- inner_join(counties, cnty, by=c("name"="kpeds_countyoforigin"))

In [ ]:
# see counties
head(counties)

We will create the map using `ggplot` and `geom_sf` as primary functions by feeding in `counties` as the initial argument.

>Note: Text variables, like county name, can easily cause problems due to differences in format: all uppercase, all lowercase, or camelcase.  An easy solution is to convert everything to the same format as demonstrated above. It is also helpful to get a count of unique names in a dataframe before and after a join to check for errors.

In [ ]:
# Create the plot
ggplot(counties) +                                              # insert the name of the main dataset here
    geom_sf(aes(fill=n), color='white') +                       # in the fill parameter use "count" variable
    scale_fill_gradient(low="light blue",high="red") +          # choose colors for the gradient
    geom_text(aes(x=long, y=lat, label=name), size=2) +         # add county names as labels using centroids defined in the table
    labs(
        title = "More graduates are originally from the [redacted] of Kentucky",
        caption = "Source: KPEDS Data"
    ) +
    theme_minimal()

>**Note**: This heatmap shows one of the issues when displaying people by county in Kentucky. Out of 120 counties in the state, the population of just two (Jefferson County and Fayette County) comprise over 20% of the population of the state as a whole. Heatmaps on counts will tend to compress the scale for the remaining 118 counties. One solution is to use percentages based on some consistent metric, whether that be high school graduates, overall population within a certain age range, or enrollees from that area. Each of these would be presenting a different story and the denominator should be selected with care.

### Percentage of graduates in the 7 Major Groups by Appalachian status

It may also be useful for us to visualize the differences in the percentage of graduates by major goups (used in the Postsecondary Feedback Report) based on the geographic location of the postsecondary institution. Bar plots can aid visual inspection of group differences as well as provide an effective communication tool for the outside audience (for example, if you would like to highlight a significant difference between groups as we do in the example below).

Recall our seven major groupings column in df, `deg_class`.

In [ ]:
# what were the names of those deg_class categories again?
unique(df$deg_class) 

We will also need our handy institution crosswalk `kpeds_inst_xwalk`, which provides the Applachian status of each institution's primary campus location.

In [ ]:
# read institution crosswalk into r
qry <- "
select inst_code, wib, appalachian
from kystats_2020.kpeds_inst_xwalk
"
inst_xwalk <- dbGetQuery(con, qry)

In [ ]:
# can match on the institution code
df_app <- df %>% 
    left_join(inst_xwalk, by=c("kpeds_institution" = "inst_code"))

From here, we can feed our data frame of the percentage of graduates in each degree field directly into our `ggplot()` call.

> We can plot a side by side barplot by setting `position = position_dodge()`.

In [ ]:
# plot percentages by degree group and appalachian status
df_app %>%
    count(deg_class, appalachian) %>%
    group_by(appalachian) %>%
    mutate(percent = (n/sum(n)) * 100) %>% #up to this point, we have been doing some classic dplyr work as introduced in the data exploration notebook
    ggplot(aes(x=word(deg_class,1), y=percent, fill=as.factor(appalachian))) +
    geom_bar(stat="identity", position=position_dodge()) +
    labs(
        title = 'Graduates from Appalachian Institutions do not receive [redacted] degrees as often',
        y = 'Percentage of graduates',
        x='Degree Field',
        fill = 'Appalachian Status',
        caption = 'Source: KPEDS data'
    ) +
    theme_minimal() +
    ylim(0,100)

<font color=red><h3> Checkpoint 2: World Wide WIB </h3></font> 

Recreate this bar plot broken down by WIB.
>Note: we have not introduced faceting, but feel free to look it up if you want each wib to display as a separate plot: `+ facet_wrap(~wib)`. 

## Employment outcomes

Similar to how we visualized descriptive statistics to better understand the graduates within our cohort, we can use similar techniques to visualize descriptive statistics regarding our cohort's earnings and employment outcomes. In this section, we will walk through three visualizations:
- Quarterly wages distribution via density plots by degree rank (`geom_density()` and `facet_grid()`)
- A bar plot of Industry breakdowns by Appalchian status (`geom_bar()`)
- A heatmap of employment patterns by cohort (`geom_tile()`)

First, we need to read our employment outcomes for our cohort, which are stored in `cohort_wages`, into R.
>Note: By now, you have probably noticed the utility of `glimpse()`.  Other functions you may find useful include: `head()`, `summary()`, `names()`, and `unique(df$variable)`.

In [ ]:
# read into R
qry = "
select *
from ada_ky_20.cohort_wages
"
df_wages <- dbGetQuery(con, qry)

In [ ]:
# see df_wages
glimpse(df_wages)

### Distribution of quarterly wages by degree rank

We can use a density plot (a smoothed version of a histogram) to visualize distributions of wages by degree rank in our cohort. 

When we want to compare multiple groups in one plot, there is a high chance that the plot will become overcrowded. A good solution for such case is using small multiples (a series of mini-graphs for each group which use the same scale and axes). In this example we will visualize the quarterly wage distribution for graduates by degree rank using mini-density plots for each degree rank.

As before, we can find a rough distribution of all quarterly wages using our code from the Data Exploration notebook.

In [ ]:
# distribution of quarterly wages
df_wages %>%
    group_by(coleridge_id, job_date) %>%         #recall that job date was a created variable that combines year and quarter into a single date field
    summarize(quarterly_wages = sum(wages)) %>%  #sums wages across all jobs held by a single person during this quarter
    ungroup() %>%
    select(quarterly_wages) %>%
    summary()

In plotting, we will add in a grouping variable, `degreerank`, in our `group_by()` function. This will allow us to create separate grids based on the `degreerank` level.

> Feel free to change the limit on the x-axis.

In [ ]:
# plot
df_wages %>%
    group_by(coleridge_id, job_date, degreerank) %>%
    summarize(quarterly_wages = sum(wages)) %>%
    ungroup() %>%
    ggplot(aes(x=quarterly_wages)) +
    geom_density() + 
    facet_grid(degreerank ~ .) +
    labs(
        title = 'Quarterly wages tend to [redacted] as the degree level [redacted]',
        y = 'Density',
        x='Quarterly Wages',
        caption = 'Source: KPEDS and KY UI wage records data'
    ) +
    theme_minimal() +
    xlim(0, 25000)

<font color=red><h3> Checkpoint 3: Working with `facet_grid()` </h3></font> 

Create a small multiples visualization with your own variables of interest.
>Note: `facet_grid` and `facet_wrap` can frequently accomplish the same thing.  `facet_grid` is generally superior when trying to show some outcome split by two characteristics.  One example might be splitting wages by gender (found in `master_person`) and degree STEM classification (variable `kpeds_isstem` in the `kpeds_degree` table). Although these variables are not part of the current dataset, they could easily be added using the skills introduced in the data exploration notebook.

### Key Sectors by Institution Location

We can also inspect where graduates from institutions from different locations ended up by comparing the percentages of graduates employed in the five key sectors by Appalachian status. First, let's go ahead and create a new column `key_sect` to represent these sectors.

In [ ]:
# add in five key sectors
# changed names a bit for viz
df_wages <- df_wages %>%
    mutate(key_sect = case_when(
        majorindustry == "Manufacturing" ~ "Manufacturing",
        majorindustry == "Construction" ~ "Construction",
        majorindustry == "Health Care and Social Assistance" ~ "Health Sciences",
        majorindustry == "Transportation and Warehousing" ~ "Transportation",
         majorindustry %in% c("Professional, Scientific, and Technical Services",
                             "Finance and Insurance", 
                             "Information",
                             "Wholesale Trade") ~ "Business",
        TRUE ~ "Non_Key"
    )
          )


Then we can match our graduates in `df_wages` by their institution to find their Appalachian status.

In [ ]:
# can match on the institution code
df_wages_app <- df_wages %>% 
    left_join(inst_xwalk, by=c("kpeds_institution" = "inst_code"))

Before we can graph the percentage of graduates in each of the sectors grouped by the Appalachian status of their institution, since an individual may have multiple rows of employment based on their employment history in their first year post-graduation, we must find the number of unique individuals, i.e. the number of individuals who received any wages in Kentucky, broken down by the location of the institution of their most recent graduation. We will assign this output to `app_break`.

In [ ]:
# find breakdown of employed graduates in KY by appalachian status of institution
app_break<-df_wages_app %>% 
    group_by(appalachian) %>%
    summarize(n_total=n_distinct(coleridge_id))

As we did in the Data Exploration notebook, we can proceed by finding the percentage of graduates in each of the sectors grouped by the Appalachian status of their institution, and then use those counts as an input to `ggplot()`. To find the percentage of individuals employed in the sector in at least one quarter, we will divide these counts by those in `app_break`.

In [ ]:
# bar plot of percentages by industry and appalachian status
df_wages_app %>%
    group_by(key_sect, appalachian) %>%
    summarize(n = n_distinct(coleridge_id)) %>%
    ungroup() %>%
    left_join(app_break, "appalachian") %>%
    mutate(pct = (n/n_total)*100) %>%    
    ggplot(aes(x=word(key_sect, 1), y=pct, fill=as.factor(appalachian))) +
    geom_bar(stat="identity", position=position_dodge()) +
    labs(
        title = 'Graduates from Appalachian Institutions were more likely to end up in [redacted]',
        y = 'Percentage of graduates',
        x='Sector',
        fill = 'Appalachian Status',
        caption = 'Source: KPEDS and KY UI wage records data'
    ) +
    theme_minimal() +
    ylim(0,100)

<font color=red><h3> Checkpoint 4: WIB Breakdown </h3></font> 

Recreate the same bar plot by WIB status.

>Hint: You may want to leverage `facet_grid` or `facet_wrap` to make the plot easier to read. 

### Employment patterns by quarters

The final visualization in this notebook is a heatmap table of employment patterns by quarter for our cohort of Kentucky graduates. We will start by reading in the .csv file of employment patterns.

In [ ]:
patterns <- read_csv("/nfshome/INSERT_USERNAME/patterns.csv") %>% 
    select(-X1)

In [ ]:
# see patterns
patterns

In [ ]:
# Save counts to use later in the heatmap - we cannot use the counts as index, as there could be duplicate values 
counts <- patterns$count
pcts <- patterns$pct

We will add index with unique sequential numbers and remove the `count` AND `pct` columns:

In [ ]:
patterns$Pattern <- seq.int(nrow(patterns))
patterns$count <- NULL
patterns$pct <- NULL

In [ ]:
patterns

We now need to convert this table from wide to long format, since our `geom_tile()` function only works with long data frames. Instead of using `pivot_wider()` as we did to create `patterns`, we will use `pivot_longer` to create a data frame with each row corresponding to a pattern/quarter/status combination.

In [ ]:
# convert to long format
patterns_long <- pivot_longer(patterns, names_to = 'Quarter', values_to = 'Status', -c(Pattern))

In [ ]:
# see patterns_long
head(patterns_long)

Now we are ready to create the visualization using `geom_tile` in `ggplot`:

In [ ]:
# Full code for the plot

levels = ordered(c(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16))  # specify in which order to add the rows from our wide table (called "patterns") 
                                                             # we want to preserve the same ordering of rows as they are sorted in the table from highest to lowest

ggplot(data = patterns_long, aes(x = Quarter, y = ordered(Pattern, levels=rev(levels)))) +    # sort y-axis according to levels specified above
geom_tile(aes(fill = Status), colour = 'black') +                                             # fill the table with value from Status column, create black contouring
scale_fill_brewer(palette = "Set1") +                                                         # specify a color palette
theme(text=element_text(size=14,face="bold")) +                                               # specify font size
scale_x_discrete(position = 'top') +                                                          # include x-axis labels on top of the plot
labs(
    y = "Employment - Percentages",
    title = 'Employment Patterns by Quarters',
    caption = 'Source: KPEDS and KY UI wage records data'
) +
scale_y_discrete(labels=rev(pcts))  # rename the y-axis ticks to correspond to the counts from the table

We can also have counts on the left side of the y-axis instead.

In [ ]:
# Full code for the plot

levels = ordered(c(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16))  # specify in which order to add the rows from our wide table (called "patterns") 
                                                             # we want to preserve the same ordering of rows as they are sorted in the table from highest to lowest

ggplot(data = patterns_long, aes(x = Quarter, y = ordered(Pattern, levels=rev(levels)))) +    # sort y-axis according to levels specified above
geom_tile(aes(fill = Status), colour = 'black') +                                             # fill the table with value from Status column, create black contouring
scale_fill_brewer(palette = "Set1") +                                                         # specify a color palette
theme(text=element_text(size=14,face="bold")) +                                               # specify font size
scale_x_discrete(position = 'top') +                                                          # include x-axis labels on top of the plot
labs(
    y = "Employment - Counts",
    title = 'Employment Patterns by Quarters',
    caption = 'Source: KPEDS and KY UI wage records data'
) +                                               
scale_y_discrete(labels=rev(counts))  # rename the y-axis ticks to correspond to the counts from the table

Of course, this notebook only contains a few of the possible visualizations you can create using the `ggplot2` package. Luckily, the R community has created countless resources you can use for making all types of visualizations!